In [ ]:
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd
import time

df_all = pd.DataFrame(columns=["Title", "Authors", "Membership_lv", "Merited", "Text", "Datetime", "Original_Poster"])
count = 0

for page_i in range(0, 7960, 40):  # (start_page, end_page<<endPage * 40>>, steps<<fixed>>)  Page 1: https://bitcointalk.org/index.php?board=1.0  Page 2: https://bitcointalk.org/index.php?board=1.40
    count += 1
    if count%5 == 0:
        time.sleep(180)
    URL = "https://bitcointalk.org/index.php?board=1." + str(page_i)
    page = requests.get(URL)
    soup = BeautifulSoup(page.content, 'html.parser')
    links_with_text = []

    for random in soup.find_all("span", attrs={"id": re.compile("^msg")}):
        for b in random.find_all('a', href=True):
            
            df = pd.DataFrame(columns=["Title", "Authors", "Membership_lv", "Merited", "Text", "Datetime", "Orginal_Poster"])
            status = True
            curr_page = 0

            merited = []
            title_l = []
            authors_l = []
            date_l = []
            membership_l = []
            post_l = []
            op_l = []

            while status:
                url = b['href'][:-1] + str(curr_page)
            #     url = "https://bitcointalk.org/index.php?topic=5393078." + str(curr_page)

                result = requests.get(url)
                doc = BeautifulSoup(result.text, "html.parser")

                title = doc.title.get_text(" ")
                authors = doc.find_all("a", title=re.compile("View the profile of.*"))
                td_top = doc.find_all("td", valign="top", class_="poster_info")
                td_middle = doc.find_all("td", valign="middle")
                reply = doc.find_all("div", class_="post")

                if curr_page == 0:
                    bottom = doc.find("td", class_="middletext", valign="bottom")
                    navpages = bottom.find_all("a", class_="navPages")
                    for page in navpages:
                        if re.match("^[0-9]*$", page.string):
                            last_page = int(page.string)

                # pages
                if len(navpages) == 0 or curr_page == (last_page*20)-20:
                    status = False
                else:
                    if curr_page < (last_page*20)-20:
                        curr_page += 20

                # author
                for author in authors:
                    if not re.match("^[0-9]{10,10}$", author.string): # Filter out username 0-9
                        authors_l.append(author.string)
                        # title
                        title_l.append(title)

                # Date
                for content in td_middle:
                    try:
                        smalltext = content.find("div", class_="smalltext")
                        if smalltext.span is not None:
                            if ("January" or "February" or "March" or "April" or "May" or "June" or "July" or "August" or "September" or "October" or "November" or "December" or "Today" in str(smalltext.span)) and smalltext.span.text[-2:] in ["AM", "PM"]:
                                date_l.append(smalltext.span.get_text())

                        elif ("January" or "February" or "March" or "April" or "May" or "June" or "July" or "August" or "September" or "October" or "November" or "December" in smalltext.get_text()) and smalltext.get_text()[-2:] in ["AM", "PM"]:
                            date_l.append(smalltext.get_text())

                    except (TypeError, AttributeError):
                        continue

                # Mertied
                for content in td_middle:
                    smalltext = content.findChildren("div", class_="smalltext")
                    for t in smalltext:
                        if "Merited" in t.text:
                            total = 0
                            merited_num = re.findall('\(.*?\)',t.text)
                            for i in merited_num:
                                total += int(i[1:-1])
                            merited[len(merited)-1] = total
                        elif not re.match("^[0-9]*$", t.text):
                            merited.append(0)

                # Membership Level
                for content in td_top:
                    smalltext = content.findChildren("div", class_="smalltext")
                    for s in smalltext:
                        try:
                            if "Activity" in s.span.next_sibling.next_sibling.next_sibling.strip():
                                if s.parent.img.previous_sibling.previous_sibling.previous_sibling.strip() in ["Brand new", "Newbie", "Jr. Member", "Member", "Full Member", "Sr. Member", "Hero Member", "Legendary"]:
                                    membership_l.append(s.parent.img.previous_sibling.previous_sibling.previous_sibling.strip())
                        except AttributeError:
                            pass
                        for s_inner in s:
                            try:
                                if s_inner.string.strip() == "Guest":
                                    membership_l.append(s_inner.string.strip())                
                            except AttributeError:
                                continue

                # Reply
                for r in reply:
                    if not re.match("^[0-9]{10,10}$", r.text) or "img" in str(r): # Filter out 0-9
                        image = ""
                        if "img" in str(r):
                            images = r.findAll('img')
                            for i in images:
                                image += i['src'] + " "
                        for d in r.find_all("div"):
                            d.clear()
                            d.decompose()
                        post_l.append(image + r.get_text(" "))
                        if len(post_l) == 1:
                            op_l.append(1)
                        else:
                            op_l.append(0)
                            
                if len(membership_l) != len(authors_l):
                    for idx, lev in enumerate(membership_l):
                        if lev == "Guest":
                            authors_l.insert(idx, "Anonymous")
                            title_l.append(title)

                # # debug
                # print(len(post_l))
                # print(len(authors_l))
                # print(len(membership_l))
                # print(len(merited))
                # print(len(date_l))
                # print(len(op_l))
                # print(url)
                # if len(authors_l) != len(post_l):
                #     print(f"Error at page {curr_page}")

            df.Title = title_l
            df.Authors = authors_l
            df.Membership_lv = membership_l
            df.Merited = merited
            df.Text = post_l
            df.Datetime = date_l
            df.Orginal_Poster = op_l
            df
        df_all = pd.concat([df_all,df], ignore_index=True)
    df_all.to_csv("./bitcointalk"+ str(count) +".csv", sep=",")
